# PLATE-VS API Client Example

This notebook demonstrates how to use the Python client for the PLATE-VS API.
Make sure the PLATE-VS website is running locally on port 3000 (or adjust the base URL).

In [1]:
# Install dependencies if needed
# !pip install requests pandas

In [2]:
from platevs_client import PlateVSClient
import pandas as pd

# Initialize the client
# Adjust base_url if your local server is running elsewhere
client = PlateVSClient(base_url="http://localhost:3000")

print(f"Client initialized for {client.base_url}")

## 1. Check Service Status

In [3]:
# Check if PLATE-VS services are accessible
status = client.check_service_status()
print("Service Status:")
for service, is_available in status.items():
    print(f"  {service}: {'Available' if is_available else 'Unavailable'}")

Service Status:
  main: Available
  api: Available


## 2. Search by UniProt ID

In [4]:
# Example: Search for EGFR (P00533)
uniprot_id = "P00533"
print(f"Searching for {uniprot_id}...")

# Get ligands as DataFrame
df_ligands = client.get_protein_ligands(uniprot_id)
if not df_ligands.empty:
    print(f"Found {len(df_ligands)} ligands")
    display(df_ligands.head())
else:
    print("No ligands found")

Searching for P00533...
Found 1859 ligands


,id,protein_id,molecule_chembl_id,standard_type,standard_value,standard_units,pchembl_value,activity_comment,assay_chembl_id,confidence_score,target_chembl_id,document_chembl_id
0,41761,P00533,CHEMBL68920,IC50,300.0,nM,6.52,NaN,CHEMBL621151,NaN,CHEMBL203,CHEMBL1134862
1,41762,P00533,CHEMBL68920,IC50,7820.0,nM,5.11,NaN,CHEMBL615325,NaN,CHEMBL203,CHEMBL1134862
2,41763,P00533,CHEMBL69960,IC50,40.0,nM,7.40,NaN,CHEMBL621151,NaN,CHEMBL203,CHEMBL1134862
3,41764,P00533,CHEMBL69960,IC50,440.0,nM,6.36,NaN,CHEMBL615325,NaN,CHEMBL203,CHEMBL1134862
4,41765,P00533,CHEMBL67057,IC50,100.0,nM,7.00,NaN,CHEMBL621151,NaN,CHEMBL203,CHEMBL1134862


## 3. Search by SMILES

In [5]:
# Example: Search for Imatinib
smiles = "Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1"
print(f"Searching for SMILES: {smiles[:30]}...")

# Similarity Search
df_sim = client.search_by_smiles(smiles, exact_match=False)
if not df_sim.empty:
    print(f"Found {len(df_sim)} similar molecules")
    display(df_sim.head())
else:
    print("No similar molecules found")

Searching for SMILES: Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC...
Found 27 similar molecules


,success,query_smiles,threshold,count,results
0,True,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,0.7,27,"{'id': 34389, 'name': 'CHEMBL941', 'smiles': '..."
1,True,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,0.7,27,"{'id': 34388, 'name': 'CHEMBL1642', 'smiles': ..."
2,True,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,0.7,27,"{'id': 147669, 'name': 'CHEMBL5925830', 'smile..."
3,True,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,0.7,27,"{'id': 147670, 'name': 'CHEMBL5995081', 'smile..."
4,True,Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc...,0.7,27,"{'id': 147668, 'name': 'CHEMBL5937667', 'smile..."


In [8]:
# Exact Match Search
smiles = "Cn1cc(S(=O)(=O)Nc2ccc(-c3ccc4c(N)n[nH]c4c3)cc2)c(C(F)(F)F)n1"
df_exact = client.search_by_smiles(smiles, exact_match=True)
if not df_exact.empty:
    print(f"Found {len(df_exact)} exact matches")
    display(df_exact.head())
else:
    print("No exact matches found")

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

## 4. Download Similarity Matrix Data

In [ ]:
# Download similarity matrix CSV for threshold 0.9
csv_path = client.download_similarity_matrix_csv(0.9, qcov_level=100)
if csv_path:
    df = pd.read_csv(csv_path)
    print(f"Downloaded {len(df)} rows")
    display(df.head())

In [ ]:
# Download SDF files (tar.gz)
sdf_path = client.download_similarity_sdf(0.9)
if sdf_path:
    print(f"Downloaded SDF archive to: {sdf_path}")

## 5. Batch Downloads

In [ ]:
# Download data for multiple thresholds
thresholds = [0.9, 0.8]
results = client.download_all_similarity_data(thresholds)

for threshold, paths in results.items():
    print(f"\nThreshold {threshold}:")
    print(f"  CSV: {paths['csv']}")
    print(f"  SDF: {paths['sdf']}")

## 6. Robust Test: Download All Similarity Matrix Data

This section demonstrates how to download the complete dataset by iterating through all available thresholds and query coverage (qcov) levels.

In [ ]:
import os
import time
import pandas as pd

# Define ranges
# Thresholds: 0.3 to 0.9
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# Qcov levels: Must be one of [50, 70, 95, 100]
qcovs = [50, 70, 95, 100]

print(f"Starting robust download test...")

total_combinations = len(thresholds) * len(qcovs)
current = 0

for threshold in thresholds:
    for qcov in qcovs:
        current += 1
        print(f"[{current}/{total_combinations}] Requesting: threshold={threshold}, qcov={qcov}")
        
        try:
            # Fetch data using the correct method
            # Note: download_similarity_matrix_csv returns a Path object or None
            csv_path = client.download_similarity_matrix_csv(similarity_threshold=threshold, qcov_level=qcov)
            
            if csv_path:
                # Verify we can read it
                df = pd.read_csv(csv_path)
                print(f"  -> Success! Downloaded to {csv_path} ({len(df)} rows)")
            else:
                print(f"  -> Failed: Client returned None")
            
        except Exception as e:
            print(f"  -> Error: {str(e)}")
        
        # Be nice to the server
        time.sleep(0.5)

print("\nRobust test completed.")